In [ ]:
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 24.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [ ]:
import torch
import pprint
import evaluate
import numpy as np

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

In [ ]:
from huggingface_hub import login


In [ ]:
# Load the dataset from Google Drive
#dataset_path = "/content/drive/MyDrive/Summarization Datasets/samsum"
#Dataset = load_dataset(dataset_path)
Dataset = load_dataset('knkarthick/dialogsum')
dataset_train = Dataset['train']
dataset_valid = Dataset['validation']
dataset_test = Dataset['test']

print(dataset_train)
print(dataset_valid)
print(dataset_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic'],
    num_rows: 12460
})
Dataset({
    features: ['id', 'dialogue', 'summary', 'topic'],
    num_rows: 500
})
Dataset({
    features: ['id', 'dialogue', 'summary', 'topic'],
    num_rows: 1500
})


In [ ]:
MODEL = 'csb05/t5-base-ami-samsum'
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 4
OUT_DIR = '/content/drive/MyDrive/Summarization Model/T5-AMI-Samsum-Dialogsum'
MAX_LENGTH = 51

In [ ]:
torch.cuda.empty_cache()

model = T5ForConditionalGeneration.from_pretrained(
    MODEL,
    use_cache=False
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

222,903,552 total parameters.
222,903,552 training parameters.


In [ ]:
tokenizer = T5Tokenizer.from_pretrained(MODEL, legacy=False)

def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['dialogue']]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )

    targets = examples['summary']
    labels = tokenizer(
        text_target=targets,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Map (num_proc=4):   0%|          | 0/12460 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=200,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=2e-5,
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

history = trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
200,1.758500,1.307136,0.478900,0.190400,0.440900,36.006000
400,1.613800,1.255547,0.495600,0.204400,0.457300,36.006000
600,1.343500,1.230798,0.503200,0.212500,0.466300,36.006000
800,1.516700,1.213491,0.506000,0.216900,0.469900,36.006000
1000,1.369200,1.205377,0.506100,0.218300,0.471100,36.006000
1200,1.228600,1.194079,0.514200,0.226900,0.480600,36.006000
1400,1.454100,1.189508,0.510000,0.225100,0.476300,36.006000
1600,1.461600,1.183854,0.515000,0.224100,0.482200,36.006000
1800,1.406000,1.179976,0.516300,0.229600,0.483700,36.006000
2000,1.351600,1.175447,0.515800,0.227500,0.481100,36.006000


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Save to Google Drive (add this after training)
trainer.save_model(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

('/content/drive/MyDrive/Summarization Model/T5-AMI-Samsum-Dialogsum/tokenizer_config.json',
 '/content/drive/MyDrive/Summarization Model/T5-AMI-Samsum-Dialogsum/special_tokens_map.json',
 '/content/drive/MyDrive/Summarization Model/T5-AMI-Samsum-Dialogsum/spiece.model',
 '/content/drive/MyDrive/Summarization Model/T5-AMI-Samsum-Dialogsum/added_tokens.json')

In [ ]:
model.save_pretrained("T5-base-ami-samsum-dialogsum")
tokenizer.save_pretrained("T5-base-ami-samsum-dialogsum")


('T5-base-ami-samsum-dialogsum/tokenizer_config.json',
 'T5-base-ami-samsum-dialogsum/special_tokens_map.json',
 'T5-base-ami-samsum-dialogsum/spiece.model',
 'T5-base-ami-samsum-dialogsum/added_tokens.json')

In [ ]:
model.push_to_hub("csb05/T5-base-ami-samsum-dialogsum")
tokenizer.push_to_hub("csb05/T5-base-ami-samsum-dialogsum")

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/csb05/T5-base-ami-samsum-dialogsum/commit/55b592deb4e0dfebe4ff8717de0cbb691c5eba45', commit_message='Upload tokenizer', commit_description='', oid='55b592deb4e0dfebe4ff8717de0cbb691c5eba45', pr_url=None, repo_url=RepoUrl('https://huggingface.co/csb05/T5-base-ami-samsum-dialogsum', endpoint='https://huggingface.co', repo_type='model', repo_id='csb05/T5-base-ami-samsum-dialogsum'), pr_revision=None, pr_num=None)